# 基于马尔可夫链的数据增强

[NLP极简数据增强资料](https://mp.weixin.qq.com/s?__biz=MzI4MDYzNzg4Mw==&mid=2247490495&idx=5&sn=a7437ee2be71d03037f5d973ccb1a3bd&chksm=ebb4236bdcc3aa7d277eca08fab7d3020103d1047620f4805dbb237be3812c785e5daf340658&mpshare=1&scene=22&srcid=&sharer_sharetime=1572928152692&sharer_shareid=da84f0d2d31380d783922b9e26cacfe2#rd)
                    
[NLP极简数据增强源码](https://github.com/leerumor/nlpcab)

In [1]:
import numpy as np
import random
import jieba
import pandas as pd
from collections import Counter
random.seed(44)

In [5]:
def toutiao_markov_augmentation(in_file, out_file):
    dat = pd.read_csv(in_file, sep="\t",encoding='utf8', header=None, names=["classname","title"])
    print(f"num train samples: {len(dat)}")
    dat = dat
    lbl2cls = {str(y[0]):x for x,y in enumerate(dat[["classname"]].drop_duplicates().values)}
    print(lbl2cls)
    print(f"num class: {len(lbl2cls)}")
    x_train, y_train = dat["title"].apply(lambda x: " ".join([w for w in jieba.cut(x, cut_all=False)]).strip()), dat["classname"]
    c = Counter(x_train.apply(lambda x: len(x.split())))
    lenghts = np.asarray(list(c.keys()))
    freq = np.asarray(list(c.values()))
    freq = freq/freq.sum()
    new_corpus, new_labels = Generator(x_train, y_train, lenghts, freq, 10, concat=False, seed=33)
    print(new_corpus[0:5])
    print(new_labels[0:5])
    with open(file=out_file,encoding="utf8",mode= "w") as fout:
        for i in range(len(new_corpus)):
            print("new_labels[i]:{0}".format(new_labels[i]))
            print("lbl2cls[new_labels[i]]:{0}".format(lbl2cls[new_labels[i]]))
            fout.write("\t".join([new_labels[i], new_corpus[i]])+"\n")
    print(f"save output to {out_file}")

### DEFINE MARKOV CHAIN GENERATOR ###

def build_chain(texts):
    
    index = 1
    chain = {}
    
    for text in texts:
        
        text = text.split()
        for word in text[index:]:
            key = text[index-1]
            if key in chain:
                chain[key].append(word)
            else:
                chain[key] = [word]
            index += 1
        
        index = 1
    
    return chain

def create_sentence(chain, lenght, seed):
    
    np.random.seed(seed)
    
    start = random.choice(list(chain.keys()))
    text = [start]

    while len(text) < lenght:
        try:
            after = random.choice(chain[start])
            start = after
            text.append(after)
        except: #end of the sentence
            #text.append('.')
            start = random.choice(list(chain.keys()))
    
    return ' '.join(text)

def Generator(x_train, y_train, lenghts, freq, rep, concat=False, seed=33):
    
    np.random.seed(seed)
    
    new_corpus, new_labels = [], []
    
    for i,lab in enumerate(np.unique(y_train)):

        selected = x_train[y_train == lab]
        chain = build_chain(selected)

        sentences = []
        for i in range(rep):
            lenght = int(np.random.choice(lenghts, 1, p=freq))
            sentences.append(create_sentence(chain, lenght, seed))

        new_corpus.extend(sentences)
        new_labels.extend([lab]*rep)
    
    if concat:
        return list(x_train) + new_corpus, list(y_train) + new_labels
    
    return new_corpus, new_labels

In [6]:
def main():
    toutiao_markov_augmentation("F:\\document\\datasets\\nlpData\\text_classifier_data\\THUCNews_ch\\cnews.train.txt", "aug_train.txt")
    print("done.")

In [7]:
main()

num train samples: 50000
{'体育': 0, '娱乐': 1, '家居': 2, '房产': 3, '教育': 4, '时尚': 5, '时政': 6, '游戏': 7, '科技': 8, '财经': 9}
num class: 10
['今非昔比 。 ” 。 本 周末 时 ， 阿联 自己 的 塞 弗罗 曼 的 儿子 来 证明 自己 也 没有 第三种 得分 … 丹佛 高原 ， 今天 就是 这种 全面 表现 无愧 第一 中锋 受 待遇 ， 双方 战成 49 - 莫雷 和 李冬生 和 王 吉恩 报道 ， 所以 他 都 在 2011 - 约翰逊 和 贾马尔 - 埃文斯 ， “ 大 前锋 拉里 - 马丁 今天 还 可能 要求 很 好 机会 打败 爱国者 队 的 采访 时 表示 ： 雷 - 邓肯 中投 偏出 ， 他们 何时能 到头 了 一些 伟大 的 。 可以 看出 在 三分 。 ” 吉尔伯特 - 21 分 ， 麦克戴斯 13 - 布鲁克斯 伤停 之后 ， 森林狼 。 如今 的 祭旗 ， ” 格里芬 是 雄鹿 和 热情 ， 他 说 两句 了 两次 罚球 17 分 ， 球队 ， 华仔 失去 悬念 ， 本周 升幅 最大 的 数据 媲美 的 肖恩 - 刘易斯 埋伏 的 真实 意义 。 湖人队 中 ， 这样 的 复健 运动 中 所 希望 。 而 以 62 - 戈登 三分 。 骑士 多项 队史 新高 。 公牛 被 判罚 毫 无疑 都 觉得 很 高兴 过头 了 。 [ 直播 中 得到 19 分 ， 但是 火箭 对 球队 才能 在 西部 前八 。 最近 三场 比赛 后 双手 暴扣 ， 锁定 胜局 。 至于 小 詹姆斯 这样 的 ， 沃尔 自己 出手 仍然 领先 。 黄蜂 等 筹码 的 一笔 超级 球星 德隆 特 、 灰熊 的 光彩 相当 罕见 公开批评 全队 第四 好 地 冲向 篮筐 上 双 20 分 ， 会 在 土耳其 联赛 第一轮 的 ， 巴蒂尔 在 击败 的 控球 ， 结果 ， 我们 认识 ， 高居 全 联盟 顶级 球员 ； 快船 能够 挖 来 了 。 吕晓明 在 盐湖城 等等 。 12 日 ( 韦德 和 火箭 最 重要 的 卫士 or 阿伦 因为 上门 求购 新浪 体育讯 北京 时间 11 月 7 篮板 ， 怎么 说 ， 以 102 击败 湖人 双 的 成功